## datasets

- `DatasetDict`
    - `Dataset`
        - 可支持索引

In [1]:
from datasets import load_dataset
clinc = load_dataset("clinc_oos", "plus")

Found cached dataset clinc_oos (/home/whaow/.cache/huggingface/datasets/clinc_oos/plus/1.0.0/abcc41d382f8137f039adc747af44714941e8196e845dfbdd8ae7a7e020e6ba1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
type(clinc)

datasets.dataset_dict.DatasetDict

In [30]:
clinc

DatasetDict({
    train: Dataset({
        features: ['text', 'intent'],
        num_rows: 15250
    })
    validation: Dataset({
        features: ['text', 'intent'],
        num_rows: 3100
    })
    test: Dataset({
        features: ['text', 'intent'],
        num_rows: 5500
    })
})

In [4]:
clinc['train']

Dataset({
    features: ['text', 'intent'],
    num_rows: 15250
})

In [5]:
clinc['train'][42]

{'text': 'what phrase means goodbye in hawaii', 'intent': 61}

In [17]:
# clinc['train'].features

In [14]:
clinc['train'].features['intent'].int2str(61)

'translate'

In [29]:
# 维护类别id及其语义信息
# intents = clinc['train'].features['intent']
# intents = clinc['test'].features['intent']
intents = clinc['validation'].features['intent']
print(intents.int2str(61))
print(intents.str2int(intents.int2str(61)))

translate
61


## metrics

In [18]:
from datasets import load_metric

In [20]:
import os
os.environ['HTTP_PROXY'] = 'http://127.0.0.1:7890'
os.environ['HTTPS_PROXY'] = 'http://127.0.0.1:7890'

In [21]:
accuracy_score = load_metric('accuracy')


In [32]:
# accuracy_score
# references: true labels;
accuracy_score.compute(predictions=[1, 1, 2, 2], references=[2, 1, 2, 1])

{'accuracy': 0.5}

### glue metric

> General Language Understanding Evaluation (GLUE) benchmark

The **M**icrosoft **R**esearch **P**araphrase **C**orpus (Dolan & Brockett, 2005) is a corpus of sentence pairs automatically extracted from online news sources, with human annotations for whether the sentences in the pair are semantically equivalent.

In [ ]:
mrpc_metric = load_metric("glue", "mrpc")
sst2_metric = load_metric('glue', 'sst2') 

## models

In [34]:
# model.state_dict()
# model.state_dict().items()

# (key, value)
# list(model.state_dict().items())[42]